In [6]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import timeit
from models import *
from equations import *
from solvers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_file_name = "data/data_2017.npz"
params = ModelParams.load_from_npz(data_file_name)
params.gamma = np.swapaxes(params.gamma, 1, 2)
model = Model(params)
shocks = model.shocks

In [8]:
w_hat = model.sol.w_hat
P_hat = model.sol.P_hat
(N, S, alpha, beta, gamma, theta, pi, tilde_tau, X, V, D, countries, sectors) = params.unpack()
(lambda_hat, d_hat, tilde_tau_hat) = shocks.unpack()
(c_hat, P_hat, pi_hat, X_prime, D_prime, p_index, real_w) = generate_equilibrium(
    w_hat, P_hat, alpha, beta, gamma, theta, pi, tilde_tau, V, D, X, lambda_hat, d_hat, tilde_tau_hat)

In [10]:
X_prime

array([[4.82759599e+07, 4.75723482e+07, 9.44606920e+07, 2.28340904e+07,
        4.21222878e+07, 1.03402718e+08, 6.81634038e+07, 4.73307454e+07,
        4.78348295e+07, 1.27383248e+07, 3.86864645e+07, 2.25355491e+08,
        3.87891074e+06, 1.08866955e+08, 7.20174126e+07, 8.61373535e+07,
        1.58513399e+08, 7.09627640e+07, 8.27517054e+08, 1.23141887e+08,
        3.64231478e+08, 2.59827366e+06, 4.99355194e+06],
       [1.09608430e+07, 7.64267945e+06, 2.63109927e+07, 1.14600895e+07,
        3.14176950e+07, 3.91338225e+07, 2.28453447e+07, 5.63769026e+07,
        2.40515079e+07, 1.09902273e+07, 2.37901509e+07, 6.11813448e+07,
        1.33542725e+07, 3.94036373e+07, 3.12431911e+07, 3.50084239e+07,
        4.93140917e+07, 1.93239938e+07, 1.85291982e+08, 3.39974395e+07,
        9.07773161e+07, 7.25141772e+05, 3.06339667e+06],
       [1.21952663e+07, 9.40361612e+06, 5.56827454e+07, 1.86191987e+07,
        2.16856475e+07, 6.67141710e+07, 2.62137867e+07, 6.80979119e+07,
        4.40597192e+07

In [14]:
# solver = ModelSolver(model)
# solver.solve()

In [5]:
model.sol.w_hat

array([0.97033871, 0.96911607, 0.97387002, 0.99820079, 0.95593071,
       0.97606713, 0.98658432, 0.97576458, 0.96835557, 0.99024819,
       0.98648731, 0.97833887, 0.96259532, 0.96782096, 0.98152818,
       1.01006971, 0.9860503 , 0.95696931, 0.97516203, 0.97440822,
       1.03528361, 0.98272038, 0.97339196, 0.97339416, 0.97102776,
       0.99308295, 0.96569517, 0.98838707, 0.9764247 , 0.97497409,
       0.97841698, 0.99760393, 0.97766159, 0.98855539, 1.03775788,
       0.97631686, 1.00836504])